# 5-3-H Спички детям не игрушка!

## Условие
Вася любит решать головоломки со спичками. Чаще всего они формулируется следующим образом: дано изображение $A$, составленное из спичек; переложите в нем минимальное количество спичек так, чтобы получилось изображение $B$.

Головоломки, которые решает Вася, всегда имеют решение. Это значит, что набор спичек, используемый в изображении $A$, совпадает с набором спичек, используемым в изображении $B$. Кроме того, в одном изображении никогда не встречаются две спички, у которых есть общий участок ненулевой длины (то есть спички могут пересекаться, но не могут накладываться друг на друга).
Вася устал решать головоломки вручную, и теперь он просит вас написать, программу, которая будет решать головоломки за него. Программа будет получать описания изображений $A$ и $B$ и должна найти минимальное количество спичек, которые надо переложить в изображении $A$, чтобы полученная картинка получалась из $B$ параллельным переносом.

### Формат ввода

В первой строке входного файла содержится целое число $n$ — количество спичек в каждом из изображений ($1 \leq n \leq 1000$). В следующих $n$ строках записаны координаты концов спичек на изображении $A$. Спичка номер $i$ описывается целыми числами $x_{1i}, y_{1i}, x_{2i}, y_{2i}$ — координатами ее концов. Следующие $n$ строк содержат описание изображения $B$ в таком же формате. Набор длин этих спичек совпадает с набором длин спичек с изображения $A$.
Все координаты по абсолютной величине не превосходят $10^4$. Все спички имеют ненулевую длину, то есть $x_{1i} \neq x_{2i}$ или $y_{1i} \neq y_{2i}$.

### Формат вывода

Выведите минимальное количество спичек, которые следует переложить, чтобы изображение $A$ совпало с изображением $B$, с точностью до параллельного переноса.

### Пример

**Входной файл**:
```txt
6
2 7 1 5
3 5 2 7
3 5 1 5
1 5 1 3
3 3 3 5
1 3 3 3
7 7 9 7
9 7 11 7
9 7 9 5
8 3 9 5
9 5 10 3
10 3 8 3
```
**Выход**:
```txt
3
```

<center><img src="images/matches_1.jpeg" width=550px></center>
<center><b>Рис. 1.</b> Пример: требуется переложить минимум 3 спички</center>


## Идеи по решению

1. Спички могут быть разной длины.

2. Чтобы изображения совпали **с точностью до параллельного переноса**, важны не абсолютные координаты спичек, а только взаимное расположение спичек внутри изображения.

3. Если **поменять местами начало и конец спички** (то есть повернуть ее на 180 градусов относительно центра), то это будет та же самая спичка и ходы на такой поворот не расходуются (рис. 2). Данный вывод делается из анализа приведенных Яндексом примеров.
   
   Имеет смысл уже на этапе ввода данных "поворачивать" спички в соответствии с некоторым правилом: например, пусть конец спички смотрит вправо; либо пусть конец смотрит вверх, если спичка расположена строго вертикально.

   <br>
   
   <center><img src="images/matches_2.jpeg" width=300px></center>
   <center><b>Рис. 2.</b> Здесь требуется 0 ходов: фигуры эквивалентны</center>

   <br>

4. Можно считать, что спичка осталась непереложенной, если она ориентирована в изображеннии $A$ так же как в изображении $B$ (рис. 3). Вычисляем смещение для начальных точек спичек и соответственно для конечных точек спичек: 
   $$\text{offset}_0 = (x_0^b - x_0^a, y_0^b - y_0^a)$$
   $$\text{offset}_1 = (x_1^b - x_1^a, y_1^b - y_1^a)$$
   Если $\text{offset}_0 = \text{offset}_1$, то спичка может считаться непереложенной.

   <br>

   <center><img src="images/matches_3_offset.jpeg" width=500px></center>
   <center><b>Рис. 3.</b> Спичка может считаться непереложенной</center>

   <br>

5. Ключевая идея – все пары спичек $a_i : b_j$, имеющие одинаковое смещение, можно считать непереложенной частью рисунка. Соответственно, чем больше мы найдем таких пар, тем меньше будет искомое количество спичек, которые необходимо переложить.

   На рис. 4: пары точек $a_1: b_2$, $a_2: b_3$ и $a_3: b_4$ имеют одинаковое смещение. А все спички, не входящие в эти пары – переложены. 

   <br>

   <center><img src="images/matches_4.jpeg" width=500px></center>
   <center><b>Рис. 4.</b></center>

   <br>

## Алгоритм

1. С помощью вложенных циклов вычисляем смещения для всех пар спичек $a_i: b_j$

2. Если для конкретной пары спичек смещения начальных и конечных точек равны, добавляем такое смещение в словарь-счетчик (будем использовать для этих целей объект класса `Counter` из модуля `collections`)

3. Берем наибольшее количество найденных одинаковых смещений и вычитаем его из общего числа спичек – ответ готов!

## Код на Python

In [3]:
from collections import Counter

with open('input/input-5-3-H.txt', mode='r') as f:

    n = int(f.readline()) # количество спичек

    # По очереди для фигур А и B читаем данные 
    # и сразу контролируем направление спичек
    a, b = [], []
    for fig in [a, b]:
        for _ in range(n):
            m = list(map(int, f.readline().split()))
            start = (m[0], m[1])
            end = (m[2], m[3])
            # Унифицируем направление спички: пусть конец спички смотрит вправо, 
            # либо вверх, если спичка лежит строго вертикально
            if start[1] > end[1] or (start[1] == end[1] and end[0] < start[0]):
                start, end = end, start
            fig.append((start, end))

# Счетчик смещений, встретившихся при сравнении изображений
res_counter = Counter()

# Попарно сравниваем спички из первого и второго изображений 
for m_a in a:
    for m_b in b:
        #  Вычисляем смещение
        offset_0 = (m_b[0][0] - m_a[0][0], m_b[0][1] - m_a[0][1]) # для начальных точек
        offset_1 = (m_b[1][0] - m_a[1][0], m_b[1][1] - m_a[1][1]) # и для конечных точек
        
        # Если смещения начальных и конечных точек равны,
        # значит спички эквивалентны – добавляем смещение в счетчик
        if offset_0 == offset_1:
            res_counter[offset_0] += 1

# Наиболее частое смещение, позволяет нам считать непереложенными спички,
# которые это смещение дают.
# Если же не нашлось ни одной пары эквивалентных спичек, 
# придется переложить их все
same = res_counter.most_common(1)[0][1] if len(res_counter) else 0

print(n - same)

3
